In [2]:
import os
os.chdir('../')  # Moving up one directory to the root
os.environ['WRAPT_DISABLE_EXTENSIONS'] = 'true'
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from main import SentimentAnalyzer
from config.model_config import ModelConfig
from models.sentiment_model import EnhancedDistilBertForSentiment, ModelTrainer
from models.modelPersistence import ModelPersistence
from utils.dataVisualizer import DataVisualizer
from utils.modelEvaluator import ModelEvaluator

2024-12-12 12:06:47.623981: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Transformers package location: /opt/anaconda3/lib/python3.12/site-packages/transformers/__init__.py


In [3]:
# Cell 3: Data Preparation
analyzer = SentimentAnalyzer()
analyzer.process_data()

train_texts = analyzer.train_texts
val_texts = analyzer.val_texts
test_texts = analyzer.test_texts
train_labels = analyzer.train_labels
val_labels = analyzer.val_labels
test_labels = analyzer.test_labels

Training set size: 2100 (70.0%)
Validation set size: 600 (20.0%)
Test set size: 300 (10.0%)


In [4]:
analyzer.process_data()
analyzer.initialize_model()

# model configuration
print("Model Configuration:")
print(f"BERT Model: {ModelConfig.BERT_MODEL}")
print(f"Learning Rate: {ModelConfig.LEARNING_RATE}")
print(f"Batch Size: {ModelConfig.BATCH_SIZE}")
print(f"Max Length: {ModelConfig.MAX_LENGTH}")
print("done")

Training set size: 2100 (70.0%)
Validation set size: 600 (20.0%)
Test set size: 300 (10.0%)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model Configuration:
BERT Model: distilbert-base-uncased
Learning Rate: 5e-06
Batch Size: 32
Max Length: 256
done


In [5]:
# Cell 5: Model Architecture Verification
analyzer.verify_model_architecture()


Model Architecture Summary:
Model: "enhanced_distil_bert_for_sentiment"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_distil_bert_model (TFDi  multiple                  66362880  
 stilBertModel)                                                  
                                                                 
 bidirectional (Bidirection  multiple                  426496    
 al)                                                             
                                                                 
 dense (Dense)               multiple                  4128      
                                                                 
 multi_head_attention (Mult  multiple                  16800     
 iHeadAttention)                                                 
                                                                 
 dense_1 (Dense)             multiple                  4128      
   

In [ ]:
# Cell 6: Training and Visualization
# Train model
history = analyzer.trainer.train(analyzer.train_dataset, analyzer.val_dataset)


Epoch 1/10
16/66 [======>.......................] - ETA: 13:00 - loss: 1.0967 - accuracy: 0.3516

In [ ]:
# Initialize evaluator
evaluator = ModelEvaluator(model=analyzer.model, trainer=analyzer.trainer)

# Run evaluation
eval_results = evaluator.evaluate_model(
    texts=analyzer.test_texts[:500],
    labels=analyzer.test_labels,
    batch_size=32
)

# Visualize results
evaluator.visualize_results(eval_results)

In [ ]:
# Cell 7: Save Model
modelPersistence = ModelPersistence()
modelPersistence.save_model(epoch=4, history=history)